In [2]:
#!pip install ipywidgets
#!pip install opencv-python-headless  # Avoid unnecessary GUI dependencies in Colab
import cv2
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
from torchvision.io import read_video
from re import I
import os
import torch
from torchvision.io import read_video
from IPython.display import HTML, display
from base64 import b64encode
from PIL import Image
import io
from sklearn.model_selection import train_test_split

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:

def extract_frames(video_path, every_n_frames=1):
    """Extracts frames from a video file."""
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frames = []
    for i in range(frame_count):
        ret, frame = cap.read()
        if i % every_n_frames == 0 and ret:
            frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    cap.release()
    return frames
from IPython.display import display, clear_output, HTML

# Create a display handle
display_handle = display(display_id=True)

In [4]:
df = pd.read_csv(r'/content/drive/MyDrive/VIZ.AI project/EchoNet-Dynamic/FileList.csv')
df_tracing = pd.read_csv(r'/content/drive/MyDrive/VIZ.AI project/EchoNet-Dynamic/VolumeTracings.csv')

In [5]:
df_tracing['deltaX'] = (df_tracing['X2'] - df_tracing['X1']) / 10
df_tracing['deltaY'] = (df_tracing['Y2'] - df_tracing['Y1']) / 10
df_tracing['slope'] = (df_tracing['deltaY'] / df_tracing['deltaX'])
df_tracing['length'] = (df_tracing['deltaX']**2 + df_tracing['deltaY']**2)**0.5
gb_tracing = df_tracing.groupby(['FileName', 'Frame'])
df_tracing['number of disks'] = gb_tracing['length'].transform('count') - 1
df_tracing['deltaH'] = gb_tracing['length'].transform('first') / df_tracing['number of disks']
df_tracing['deltaV'] = np.pi * ((df_tracing['length'] / 2)**2) * df_tracing['deltaH']
df_tracing['first deltav'] = gb_tracing['deltaV'].transform('first')
df_tracing['Volume'] = gb_tracing['deltaV'].transform('sum') - df_tracing['first deltav']
df_volumes = gb_tracing['Volume'].first().to_frame().reset_index()
df_volumes.loc[df_volumes.groupby('FileName')['Volume'].idxmax(), 'Label'] = 'EDV'
df_volumes.loc[df_volumes.groupby('FileName')['Volume'].idxmin(), 'Label'] = 'ESV'
# df_volumes['EDV frame'] = df_volumes.loc[df_volumes.groupby('FileName')['Volume'].idxmax(), 'Frame']
# df_volumes['EDV frame'] = df_volumes['EDV frame'].ffill()
# df_volumes['ESV frame'] = df_volumes.loc[df_volumes.groupby('FileName')['Volume'].idxmin(), 'Frame']
# df_volumes['ESV frame'] = df_volumes['ESV frame'].bfill()
# df_volumes = df_volumes.drop_duplicates(subset='FileName')
# df_volumes['EDV frame'] = df_volumes['EDV frame'].astype(int)
# df_volumes['ESV frame'] = df_volumes['ESV frame'].astype(int)
print(df_volumes)
# print(df_tracing.loc[df_tracing['FileName'] == '0X710EC36F325037EE.avi', 'Volume'])
df_volumes.to_csv('Frames labels.csv', index=False)

                     FileName  Frame     Volume Label
0      0X100009310A3BD7FC.avi     46  24.002426   EDV
1      0X100009310A3BD7FC.avi     61   5.411705   ESV
2      0X1002E8FBACD08477.avi      3  20.289618   EDV
3      0X1002E8FBACD08477.avi     18   8.563653   ESV
4      0X1005D03EED19C65B.avi     24  11.512493   EDV
...                       ...    ...        ...   ...
20045   0XFE6E32991136338.avi     45  16.134793   ESV
20046   0XFE83FF3D3B13C3A.avi     49  13.303253   EDV
20047   0XFE83FF3D3B13C3A.avi     67   4.909705   ESV
20048   0XFEBEEFF93F6FEB9.avi     25  18.780669   EDV
20049   0XFEBEEFF93F6FEB9.avi     38  12.864623   ESV

[20050 rows x 4 columns]


In [6]:
print(df.keys())

Index(['FileName', 'EF', 'ESV', 'EDV', 'FrameHeight', 'FrameWidth', 'FPS',
       'NumberOfFrames', 'Split'],
      dtype='object')


In [7]:
videos_path = '/content/drive/MyDrive/VIZ.AI project/EchoNet-Dynamic/Videos'
EDV_train_dir = '/content/drive/MyDrive/VIZ.AI project/EchoNet-Dynamic/YOLO_data/train/EDV'
ESV_train_dir = '/content/drive/MyDrive/VIZ.AI project/EchoNet-Dynamic/YOLO_data/train/ESV'
EDV_val_dir = '/content/drive/MyDrive/VIZ.AI project/EchoNet-Dynamic/YOLO_data/val/EDV'
ESV_val_dir = '/content/drive/MyDrive/VIZ.AI project/EchoNet-Dynamic/YOLO_data/val/ESV'

df['EF_bins'] = pd.cut(df['EF'], bins=10)
df_train, df_val = train_test_split(df, test_size=0.2, stratify=df['EF_bins'], random_state=42)


# for i, filename in enumerate(df_train['FileName']):
#   if i <= 3627:
#     continue
#   print('{}){}'.format(str(i+1), filename))
#   filename_ext = filename + '.avi'
#   fullfile = os.path.join(videos_path, filename_ext)

#   try:
#     EDV_frame_idx = df_volumes.loc[(df_volumes['FileName'] == filename_ext) & (df_volumes['Label'] == 'EDV'), 'Frame'].values[0]
#   except IndexError:
#     continue
#   ESV_frame_idx = df_volumes.loc[(df_volumes['FileName'] == filename_ext) & (df_volumes['Label'] == 'ESV'), 'Frame'].values[0]
#   frames = extract_frames(fullfile)
#   if len(frames) == 0:
#     continue
#   EDV_fullfile = os.path.join(EDV_train_dir, '{} EDV.png'.format(filename))
#   ESV_fullfile = os.path.join(ESV_train_dir, '{} ESV.png'.format(filename))
#   Image.fromarray(frames[EDV_frame_idx]).save(EDV_fullfile)
#   Image.fromarray(frames[ESV_frame_idx]).save(ESV_fullfile)

for i, filename in enumerate(df_val['FileName']):
  if i <= 943:
    continue
  print('{}){}'.format(str(i+1), filename))
  filename_ext = filename + '.avi'
  fullfile = os.path.join(videos_path, filename_ext)
  try:
    EDV_frame_idx = df_volumes.loc[(df_volumes['FileName'] == filename_ext) & (df_volumes['Label'] == 'EDV'), 'Frame'].values[0]
  except IndexError:
    continue
  ESV_frame_idx = df_volumes.loc[(df_volumes['FileName'] == filename_ext) & (df_volumes['Label'] == 'ESV'), 'Frame'].values[0]
  frames = extract_frames(fullfile)
  if len(frames) == 0:
    continue
  EDV_fullfile = os.path.join(EDV_val_dir, '{} EDV.png'.format(filename))
  ESV_fullfile = os.path.join(ESV_val_dir, '{} ESV.png'.format(filename))
  Image.fromarray(frames[EDV_frame_idx]).save(EDV_fullfile)
  Image.fromarray(frames[ESV_frame_idx]).save(ESV_fullfile)


945)0X4356F73DF9D111B1
946)0X7FFB6106C779A974
947)0X7F33CFDB31ADCD6A
948)0X5496FF752FDE2035
949)0X5B7855A063C00985
950)0X32BD73DC6E258DB
951)0X7E2E8534D07DDBE
952)0X235CC08DFD8EFC4E
953)0X58FABBD1452885A9
954)0X4BFC06E24046BAC2
955)0X8A609D00A0FAB1E
956)0X4C3D1288ECBB8117
957)0X5DFC0C66C150F577
958)0XEC5FB56F6EA00B8
959)0X61109ADAC48286FB
960)0X223201B0F37DA3E1
961)0X28CA9C46619120F3
962)0X2835A031AB3FF693
963)0X564851F58C08D3CF
964)0X28259FC1DD8D4DCC
965)0X3F3DC1A6F0B18FA
966)0X6E5CF1D1178DF20B
967)0X6242E545D9E88971
968)0XF9CA1C6A7687A8D
969)0X640B3C771632C834
970)0X4AE85C73AA12C02F
971)0X3E2841AEE0E86911
972)0X8AE76DB8D580220
973)0X4D09A013097FCCB2
974)0X69AAEA04AAA0FB9D
975)0X8E7B765866B0CA7
976)0X272447C91729F139
977)0X40414596BC8DF812
978)0X7E15A2E78B07ADE2
979)0XC31D77CD4D50FA9
980)0X4527BA99A25B80AA
981)0X196C052551A4CABB
982)0X312D452CAA7E64C1
983)0X1DB488AC3583E3D6
984)0X6243CDE6C761DC2C
985)0X13381981E598316E
986)0X5E1A399B461F9D65
987)0X767947C7DECC2E0D
988)0X6C96E0DD4D10C3